In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from PIL import Image
import cv2

from copy import deepcopy

# from queue import PriorityQueue

In [ ]:
# CONSTANTS

ROBOT_WIDTH = 0.25
ROBOT_HEIGHT = 0.25

num_cols = math.ceil(2.5 / ROBOT_WIDTH) # field is 3m x 3m
num_rows = math.ceil(2.5 / ROBOT_HEIGHT)

world = np.zeros((num_rows, num_cols)) # each cell is 0.25m x 0.25m


world.shape

In [ ]:
class PriorityQueue(object):
    def __init__(self):
        self.queue = []
 
    def __str__(self):
        return ' '.join([str(i) for i in self.queue])
 
    # for checking if the queue is empty
    def isEmpty(self):
        return len(self.queue) == 0
 
    # for inserting an element in the queue
    def put(self, data):
        self.queue.append(data)
 
    # for popping an element based on Priority
    def get(self):
        try:
            max_val = 0
            for i in range(len(self.queue)):
                if self.queue[i][0] < self.queue[max_val][0]:
                    max_val = i
            item = self.queue[max_val]
            del self.queue[max_val]
            return item
        except IndexError:
            print()
            exit()

In [ ]:
class Node:
    def __init__(self, x, y):
        # numpy array is indexed by arr[y, x]
        self.x = x
        self.y = y
        self.parent = None
        self.f = 0
        self.g = 0
    
    def __str__(self):
        return "x: {} y: {}".format(self.x, self.y)
    
    def set_parent(self, n):
        self.parent = n
        
    def get_parent(self):
        return self.parent
    
    def set_f(self, f):
        self.f = f
        
    def get_f(self):
        return self.f
        
    def set_g(self, g):
        self.g = g
        
    def get_g(self):
        return self.g
    
    def get_x(self):
        return self.x
    
    def get_y(self):
        return self.y
    
    def __eq__(self, n):
        return self.x == n.get_x() and self.y == n.get_y()
    
class Map:
    def __init__(self, world_map):
        self.map = world_map # 2D numpy boolean array, 1 means occupied
        self.visited = []
        
    def visit(self, nodes):
        self.visited += nodes
        
    def get_visited(self):
        return self.visited
    
    def get_neighbors(self, n):
        """Returns unoccupied neighbor cells in the map for a given node"""
        x = n.get_x()
        y = n.get_y()
        possible_neighbors = [
            Node(x, y - 1),
            Node(x, y + 1),
            Node(x - 1, y),
            Node(x + 1, y),
            Node(x - 1, y - 1),
            Node(x - 1, y + 1),
            Node(x + 1, y - 1),
            Node(x + 1, y + 1),
        ]
        valid_neighbors = [node for node in possible_neighbors if node.get_x() >= 0 and node.get_x() < self.map.shape[1]
                              and node.get_y() >= 0 and node.get_y() < self.map.shape[0]
                              and self.map[node.get_y(), node.get_x()] == 0
                          ]
        return valid_neighbors
    
    def get_unvisited_neighbors(self, n):
        x = n.get_x()
        y = n.get_y()
        possible_neighbors = [
            Node(x, y - 1),
            Node(x, y + 1),
            Node(x - 1, y),
            Node(x + 1, y),
            Node(x - 1, y - 1),
            Node(x - 1, y + 1),
            Node(x + 1, y - 1),
            Node(x + 1, y + 1),
        ]
        valid_neighbors = [node for node in possible_neighbors if node.get_x() >= 0 and node.get_x() < self.map.shape[1]
                              and node.get_y() >= 0 and node.get_y() < self.map.shape[0]
                              and self.map[node.get_y(), node.get_x()] == 0
                          ]
        return [ne for ne in valid_neighbors if ne not in self.visited]

In [ ]:
def h(n, g):
    """
    n and g are tuple of (row, col) coordinates in world map grid
    returns euclidean distance in meters between two coordinates in map grid
    """
    y1 = n.get_y()
    x1 = n.get_x()
    
    y2 = g.get_y()
    x2 = g.get_x()
    
    d = np.sqrt((y2 - y1)**2 + (x2 - x1)**2)
    return d

In [ ]:
def bfs_coverage(start, mapp):
    
    frontier = [start]
    cur = start
    mapp.visit([cur])

    path = [cur]
    while True:
        S = [] # next level to explore
        for i in range(len(frontier)):
            node_un = mapp.get_unvisited_neighbors(frontier[i])
            S += [n for n in node_un if n not in S]
        
        if len(S) == 0:
            return path
        
        sub_path = plan_path(cur, deepcopy(S))
        path += sub_path
        mapp.visit(S)
        cur = sub_path[-1]
        frontier = S
    return path

In [ ]:
def plan_path(node, remaining_nodes):
    if len(remaining_nodes) == 0:
        return []
    else:
        d_opt = np.inf
        argmin = 0
        for i in range(len(remaining_nodes)):
            d = h(node, remaining_nodes[i]) 
            if d < d_opt:
                d_opt = d
                argmin = i
        
        closest = remaining_nodes.pop(argmin)
        return [closest] + plan_path(closest, remaining_nodes)

In [ ]:
s_n = Node(9, 9)
mapp = Map(world)

path = bfs_coverage(s_n, mapp)

In [ ]:
xs = [elem.get_x() for elem in path]
ys = [elem.get_y() for elem in path]

In [ ]:
# plt.imshow(world, cmap='gray')
# plt.scatter(xs[0], ys[0], label='start')
# plt.scatter(xs[-1], ys[-1], label='end')
# plt.plot(xs, ys)
# plt.title("Frontier Search")
# plt.legend()

In [ ]:
def coordinate_to_waypoints(coords):
    """
    turns matrix i=y, j=x values into robot waypoints
    """
    # world is 12 x 12
    waypoints = []
    for elem in coords:
        # x is correct, but y is positive in opposite direction
        w_x, w_y = elem
        w_y = world.shape[0] - 1 - w_y
        waypoint_x = w_x*(ROBOT_WIDTH) + (ROBOT_WIDTH/2)
        waypoint_y = w_y *(ROBOT_HEIGHT) + (ROBOT_HEIGHT/2)
        waypoints.append([waypoint_x, waypoint_y])
    return waypoints

In [ ]:
coordinates = np.array(list(zip(xs, ys)))

waypoints = np.array(coordinate_to_waypoints(coordinates))
# plt.figure(figsize=(3, 3))
# plt.plot(waypoints[:, 0], waypoints[:, 1])
# plt.scatter(waypoints[:, 0], waypoints[:, 1])

In [ ]:
# compression loses coverage
def compress_waypoints(wayps):
    """
    turns sequence of waypoints into smaller list
    by considering continuity in slope
    """
    denom = (wayps[1][0] - wayps[0][0])
    if denom == 0:
        prev_slope = np.inf
    else:
        prev_slope = (wayps[1][1] - wayps[0][1]) / denom
    condensed = [wayps[0]]
    
    for k in range(1, len(wayps)-1):
        wp = wayps[k]
        denom = (wayps[k+1][0] - wp[0])
        if denom == 0:
            slope_with_next = np.inf
        else:
            slope_with_next = (wayps[k+1][1] - wp[1]) / denom
    
        
        
        if slope_with_next != prev_slope:
            condensed.append(wp)
        else:
            if slope_with_next == np.inf and prev_slope == np.inf:
                prev_num = wp[1] - wayps[k-1][1]
                cur_num = wayps[k+1][1] - wp[1]
                if (prev_num < 0 and cur_num > 0) or (prev_num > 0 and cur_num < 0):
                    condensed.append(wp)
            elif slope_with_next == 0 and prev_slope == 0:
                # Just because slope is 0 (could be -dx or +dx)
                prev_denom = wp[0] - wayps[k-1][0]
                if (denom < 0 and prev_denom > 0) or (denom > 0 and prev_denom < 0):
                    condensed.append(wp)
                
        prev_slope = slope_with_next
    condensed.append(wayps[-1])
    return np.array(condensed)

In [ ]:
condensed_wps = compress_waypoints(waypoints)

# plt.figure(figsize=(3,3))
# plt.plot(condensed_wps[:, 0], condensed_wps[:, 1])
# plt.scatter(condensed_wps[:, 0], condensed_wps[:, 1])

In [ ]:
def theta_from_prev(wps):
    """
    returns theta for each waypoint which will point to
    the next waypoint
    """
    thetas = []
    for k in range(1, len(wps)):
        dy = wps[k][1] - wps[k-1][1]
        dx = wps[k][0] - wps[k-1][0]
        theta = math.atan2(dy, dx)
        thetas.append(theta)
    return np.array(thetas)

In [ ]:
# add theta dimension to waypoints
condensed_wps = np.hstack([condensed_wps, np.zeros((condensed_wps.shape[0], 1))])

# modify theta dimensions for smoother motion 
theta_wps = condensed_wps.copy()
theta_wps[1:, 2] = theta_from_prev(theta_wps[:])

# no change in theta for last waypoint necessary
# final_wps[-1, 2] = final_wps[-2, 2]

In [ ]:
# add virtual waypoints to discourage twist
trajectory = []
for i in range(len(theta_wps)-1):
    trajectory.append(theta_wps[i])
    virtual_wp = theta_wps[i].copy()
    virtual_wp[2] = theta_wps[i+1][2]
    trajectory.append(virtual_wp)
trajectory.append(theta_wps[-1])
trajectory = np.array(trajectory)

In [ ]:
plt.figure(figsize=(3, 3))
axes = plt.axes()
plt.plot(trajectory[:, 0], trajectory[:, 1])
plt.scatter(trajectory[:, 0], trajectory[:, 1])
for k in range(len(trajectory)):
#     if k == 0:
    x, y, theta = trajectory[k]
    theta = (theta + 2*np.pi) % (2*np.pi)
    y_x = np.tan(theta)
    if theta == np.pi/2:
        d_x = 0.0001
        d_y = 0.1
    elif theta == 3*np.pi/2:
        d_x = 0.0001
        d_y = -0.1
    elif theta == np.pi:
        d_x = -0.1
        d_y = 0.0001
    elif theta == 2*np.pi:
        d_x = 0.1
        d_y = 0.0001
    elif theta < np.pi/2:
        d_x = 0.1 # q1
        d_y = y_x * d_x
    elif theta > np.pi/2 and theta < np.pi:
        # q2
        d_x = -0.1
        d_y = y_x * d_x
    elif theta > np.pi and theta < 3*np.pi/2:
        d_x = -0.1 # q3
        d_y = y_x * d_x
    else:
        d_x = 0.1
        d_y = y_x * d_x

    plt.arrow(x, y, d_x, d_y, head_width=0.1, head_length = 0.1)


In [ ]:
trajectory